# Wrangling data...

In order to use your marketing data correctly it needs to be in the right format to run calculations, draw charts and marke predictions. Gettting your data in the correct order is refered to as data wrangling. 

In the section you will:

<ul>
    <li>Correcting data from Google Analytics by creating a new page URL</li>
    <li>Group data</li>
    <li>Fix data types</li>
    <li>Drop columns</li>
    <li>Replace data values</li>
    <li>Join two dataset to create a new dataset</li>
    <li>Save new datasets as CSV</li>
</ul>

In [ ]:
import pandas as pd # Lodad the pandas library

In [ ]:
#Import google analytics pages data
ga_page_data = pd.read_csv('data/ga_data/ga_pages.csv', skiprows=6, nrows=376)
ga_page_traffic_data = pd.read_csv('data/ga_data/ga_pages.csv', 
                                   skiprows=386, nrows=366, parse_dates=['Day Index'])

In [ ]:
#Importing pages data from Google Search Console
gsc_page = pd.read_csv('data/gsc_data/gsc_pages.csv')

In [ ]:
#Import Facebook ad data
fb_data = pd.read_csv('data/other_data/facebook-data-ads.csv', 
                      parse_dates=['Reporting starts','Reporting ends'])

## Fix Google Analytics Page data

We know that some of the data being gathered in Google Analytics is wrong. Google Analytics has been gathering homepage data under two page, these pages are `/` and `/www.miratrix.co.uk`.

Also during this process we will want to create a URL that can be paired with data gathered from Google Search Console. I'll split the data on '/' to allow me to break the URL into three parts. '/' denotes a breaks in the URL. Then I will rebuild the URL in the order which matches the GSC data URL's.

In [ ]:
#Look at Google Analytics data
ga_page_data.head()

### Recreate Google Analytics Page URL

In [ ]:
#Split the url by /
split_dom = ga_page_data.Page.str.rpartition("/")

In [ ]:
split_dom.head()

In [ ]:
split_dom[2].replace("", "miratrix.co.uk", inplace=True)

In [ ]:
#Change URLformat so that it matches other data sources 
split_dom[2].replace("www.miratrix.co.uk", "miratrix.co.uk", inplace=True)

In [ ]:
split_dom.head()

In [ ]:
#Build the URL and replace current URL's with new URL format
ga_page_data.Page = "https://" + split_dom[2] + split_dom[0] + "/"

In [ ]:
ga_page_data.head()

### Grouping Data

In order to fix the issue of having two homepage URL (and possibly more) we will need to group the data by Page. This poses some issues as not all the data are numerical and you are also dealing with a timeseries.

What can be done is to change the object fields to numerical and to convert the time on page column to a timedelta.

In [ ]:
ga_page_data.info()

#### Wrangling data types for Groupby function

In [ ]:
#Replace 0  with time format 00:00:00
ga_page_data['Avg. Time on Page'] = ga_page_data['Avg. Time on Page'].replace(0, '00:00:00')

In [ ]:
#Convert column from string to time delta
ga_page_data['Avg. Time on Page'] = pd.to_timedelta(ga_page_data['Avg. Time on Page'])

In [ ]:
#Replace % with nothing
ga_page_data['Bounce Rate'] = ga_page_data['Bounce Rate'].str.replace('%',"")
#Convert the column into a Float
ga_page_data['Bounce Rate'] = ga_page_data['Bounce Rate'].astype('float')

In [ ]:
#Replace % with nothing
ga_page_data['% Exit'] = ga_page_data['% Exit'].str.replace('%',"")
#Convert the column into a Float
ga_page_data['% Exit'] = ga_page_data['% Exit'].astype('float')

In [ ]:
#Replace % with nothing
ga_page_data['Page Value'] = ga_page_data['Page Value'].str.replace('£',"")
#Convert the column into a Float
ga_page_data['Page Value'] = ga_page_data['Page Value'].astype('float')

In [ ]:
#Set the index for the dataframe to Page
ga_page_data.set_index('Page',inplace=True)

In [ ]:
#Check the data to make sure the data types have changed
ga_page_data.info()

#### Applying Groupby to the data

In [ ]:
#Groupby Index and get the mean of the 'Avg. Time on Page'
avg_time = ga_page_data.groupby(ga_page_data.index)['Avg. Time on Page'].mean(numeric_only=False)

In [ ]:
#Convert avg_time to dataframe and store as ga_Time
ga_time = pd.DataFrame(avg_time)

In [ ]:
#Groupby bounce, exit, pageviews and Unique pageviews using the appropriate math 
#and store in a dataframe. Save as ga_bou_ex_type
ga_bou_ex_type = pd.DataFrame(ga_page_data.groupby(ga_page_data.index).agg({
    'Bounce Rate': ['mean'], "% Exit": ['mean'],'Pageviews': sum, 'Unique Pageviews': sum}))

In [ ]:
#Check your time groupby
ga_time['Avg. Time on Page'].head()

In [ ]:
#Check your aggreated groupby
ga_bou_ex_type.info()

#### Rebuilding the Google Analytics dataframe

Here we will rebuild the Google Analytics dataset with the fixed data for use later.

In [ ]:
#Merge ga_time and ga_bou_ex_type on page and with an inner join
new_ga = pd.merge(ga_time, #left dataset
                  ga_bou_ex_type, #right dataset
                  how='inner', # we're joning using the data in the column
                  left_on="Page", # the left dataset is neing joined on the column Page
                  right_on="Page" # the right dataset is neing joined on the column Page
                 ).sort_values(by="Avg. Time on Page" ,ascending=False) #sort the new dataframe

In [ ]:
new_ga.head()

In [ ]:
#rename (Bounce Rate, mean), (% Exit, mean), (Pageviews, sum), (Unique Pageviews, sum)
new_ga = new_ga.rename(columns={('Bounce Rate', 'mean'): 'Bounce Rate', ('% Exit', 'mean') : 'Exit',
                               ('Pageviews', 'sum') : 'Pageviews', 
                               ('Unique Pageviews', 'sum') : 'Unique Pageviews'})

In [ ]:
#Check the rename worked
new_ga.head()

## Dropping Columns

Sometimes we end up with data that we don't need that we want to drop out of our dataframe. In this tutuorial you'll learn how to drop columns from Google Analytics and Facebook ads data.

In [ ]:
#Check the Google Analytics Page data
ga_page_traffic_data.head()

In [ ]:
#Drop NaN values from the dataset
ga_page_traffic_data.dropna(axis=1, inplace=True)

In [ ]:
#Check the dropna worked
ga_page_traffic_data.head()

In [ ]:
#Check the Facebook Ads data
fb_data.head()

In [ ]:
#Drop all columns that have NaN
fb_data.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
#Check the drop worked
fb_data.head()

## Replacing missing (NaN) Facebook Ad data 

We cans see from the Facebook Ad data that there are a lot of NaN (Not A Number) values, these values are missing data. In order to visualise this data we will need to replace this data with a number. In this case we will use 0.

In [ ]:
fb_data.fillna(0,inplace=True)

In [ ]:
#Check that fillna worked
fb_data.head()

## Joining Google Analytics and Google Search Console Data

In order to extract more value from your datasets you can join them together so that you can view the trends all in one place. Pandas has various methods for joining data in this case you will use `pd.merge`.

In [ ]:
#You pandas merge to join Google Console and Google Analytics data
ga_gsc_data = pd.merge(gsc_page, #left data
                       new_ga, #right data
                       how='inner', 
                       left_on="Page", 
                       right_index=True)

In [ ]:
#Check it worked
ga_gsc_data.head()

## Saving your data to a CSV 

Saving data in pandas is super easy. We'll being using .to_csv() to save the augement and new datasets for use later.

In [ ]:
#Save the newly created Page dataset
ga_gsc_data.to_csv('data/other_data/ga_gsc_data.csv')

In [ ]:
#Save the amended facebook dataset
fb_data.to_csv('data/other_data/fixed_fb_data.csv')